In [6]:
##Import modules

import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
from scipy import stats

In [7]:
## Define Functions

####Analysis pipeline####

## ALL THAT FOLLOWS IN THIS CELL WAS CREATED BY MAX HORLBECK as seen in:
##(https://github.com/mhorlbeck/GImap_tools/blob/master/GImap_analysis.py)

# version that only filters based on cycledCol as many sgRNAs have median rep of 0 at the end
def calcLog2e_cycledonly(t0Col, cycledCol, doublesTable, filterThreshold = 1.0, pseudocount = 1.0, doublingDifferences = 1.0):
    meanCounts = pd.concat((cycledCol.groupby(doublesTable['name_a']).agg(np.median),cycledCol.groupby(doublesTable['name_b']).agg(np.median)),axis=1, keys=['a','b'])
    
    sgsToFilter = set(meanCounts.loc[meanCounts.loc[:,'b'] < filterThreshold].index).union(set(meanCounts.loc[meanCounts.loc[:,'a'] < filterThreshold].index))
    doublesTable_filt = doublesTable.loc[doublesTable.apply(lambda row: row['name_a'] not in sgsToFilter and row['name_b'] not in sgsToFilter, axis=1)]
    print(str(len(doublesTable_filt)) + ' pairs of ' + str(len(doublesTable)) + ' passing filter')
    
    countsRatio = (t0Col.loc[doublesTable_filt.index] + pseudocount).sum()*1.0/(cycledCol.loc[doublesTable_filt.index] + pseudocount).sum()
    log2es = np.log2((cycledCol.loc[doublesTable_filt.index] + pseudocount)/(t0Col.loc[doublesTable_filt.index] + pseudocount)*countsRatio)

    doubleNegatives = doublesTable.apply(lambda row: row['gene_a'] == 'negative' and row['gene_b'] == 'negative', axis=1)

    log2es -= log2es.loc[doubleNegatives].median()

    log2es /= doublingDifferences
    
    return log2es

#for a specified variable position and sgRNA, get single phenotypes, double phenotypes, and optionally single phenotype std dev.
def getXYB(sgRNA, singlePhenotypes, phenotypeMatrix, variablePosition, fixedPosition, returnXerr=False):
    if not returnXerr:
        return singlePhenotypes[variablePosition+'.mean'], \
            phenotypeMatrix.loc[sgRNA,:] if fixedPosition == 'a' else phenotypeMatrix.loc[:,sgRNA], \
            singlePhenotypes.loc[sgRNA, fixedPosition +'.mean']
    else:
        return singlePhenotypes[variablePosition+'.mean'], \
            phenotypeMatrix.loc[sgRNA,:] if fixedPosition == 'a' else phenotypeMatrix.loc[:,sgRNA], \
            singlePhenotypes.loc[sgRNA, fixedPosition +'.mean'], singlePhenotypes[variablePosition+'.std']
        
#convert phenotypes into square matrix
def generatePhenotypeMatrix(phenotypes):
    numSingles = int(np.sqrt(len(phenotypes)))
    phenotypeMatrix = np.zeros((numSingles,numSingles))
    singlesTable = []
    for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
        phenotypeMatrix[int(i/numSingles), i%numSingles] = counts
        if i%numSingles == 0:
            singlesTable.append(sgPair.split(':')[0])

    phenotypeMatrix = pd.DataFrame(phenotypeMatrix, index=singlesTable, columns=singlesTable)
    singlesTable = pd.DataFrame([s.split('_')[0] for s in singlesTable], index=singlesTable, columns=['gene'])
    
    singlePhenotypes = pd.concat((phenotypeMatrix.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanmean, axis=0), 
                                  phenotypeMatrix.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanstd, axis=0), 
                                  phenotypeMatrix.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanmean, axis=1),
                                 phenotypeMatrix.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanstd, axis=1)), 
                                 axis=1, keys=['b.mean','b.std','a.mean','a.std'])
    
    return phenotypeMatrix, singlesTable, singlePhenotypes

def abbaAveragePhenotypes(phenotypeMatrix, singlesTable):
	phenotypeMatrix_abba = (phenotypeMatrix + phenotypeMatrix.T) / 2

	singlePhenotypes_abba = pd.concat((phenotypeMatrix_abba.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanmean, axis=0), 
                                  phenotypeMatrix_abba.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanstd, axis=0), 
                                  phenotypeMatrix_abba.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanmean, axis=1),
                                  phenotypeMatrix_abba.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanstd, axis=1)), 
                                 axis=1, keys=['b.mean','b.std','a.mean','a.std'])

	return phenotypeMatrix_abba, singlePhenotypes_abba


#calculate epistasis interactions, optionally z-standardizing based on negative controls
def calculateInteractions(phenotypeMatrix, singlePhenotypes, singlesTable, fitFunction, zstandardize=True):
    emap1 = pd.DataFrame(np.zeros(phenotypeMatrix.shape), index=phenotypeMatrix.index, columns=phenotypeMatrix.columns)
    variablePosition, fixedPosition = 'a','b'
    for i, sgRNA in enumerate(phenotypeMatrix.index):
        xdata, ydata, bdata = getXYB(sgRNA, singlePhenotypes, phenotypeMatrix, variablePosition, fixedPosition)
        
        fit = fitFunction(xdata, ydata, bdata)
        epistasis = ydata - fit(xdata)

        if zstandardize:
	        emap1.loc[sgRNA,:] = epistasis / epistasis.loc[singlesTable['gene'] == 'non-targeting'].std()
       	else:
	        emap1.loc[sgRNA,:] = epistasis 

    emap2 = pd.DataFrame(np.zeros(phenotypeMatrix.shape), index=phenotypeMatrix.index, columns=phenotypeMatrix.columns)
    variablePosition, fixedPosition = 'b','a'
    for i, sgRNA in enumerate(phenotypeMatrix.index):
        xdata, ydata, bdata = getXYB(sgRNA, singlePhenotypes, phenotypeMatrix, variablePosition, fixedPosition)
        
        fit = fitFunction(xdata, ydata, bdata)
        epistasis = ydata - fit(xdata)

        if zstandardize:
	        emap2.loc[sgRNA,:] = epistasis / epistasis.loc[singlesTable['gene'] == 'non-targeting'].std()
       	else:
	        emap2.loc[sgRNA,:] = epistasis 

    emap12 = (emap1+emap2)/2
    
    emap_ave = (emap12 + emap12.T) / 2
    
    return emap1, emap2, emap_ave

#calculate all pairwise intra-sgRNA or intra-gene correlations
def calculateCorrelationMatrix(matrix, diagNull=True):
    corrMatrix = pd.DataFrame(np.corrcoef(matrix), index=matrix.index, columns=matrix.columns)
    
    if diagNull:
        for i in range(len(corrMatrix)):
            corrMatrix.iloc[i,i] = np.nan
            
    return corrMatrix

#find correlations between sgRNAs targeting the same gene and negative controls
def calculateIntrageneCorrelation(sgCorrMatrix, singlePhenotypes, singlesTable):
    sameGeneCorrTups = []
    negCorrTups = []
    for gene, sgs in singlesTable.groupby('gene'):
        for i, (sg1, row) in enumerate(sgCorrMatrix.loc[sgs.index, sgs.index].iterrows()):
            for j, (sg2, val) in enumerate(row.iteritems()):
                if i>j:
                    if gene != 'negative':
                        sameGeneCorrTups.append((sg1, sg2, 
                                                 singlePhenotypes.loc[sg1,['a.mean','b.mean']].mean(), 
                                                 singlePhenotypes.loc[sg2,['a.mean','b.mean']].mean(),
                                                val))
                    else:
                        negCorrTups.append((sg1, sg2, 
                                                 singlePhenotypes.loc[sg1,['a.mean','b.mean']].mean(), 
                                                 singlePhenotypes.loc[sg2,['a.mean','b.mean']].mean(),
                                                val))
                        
    return sameGeneCorrTups, negCorrTups


#generate a gene map by averaging sgRNA epistasis
def generateGeneMap(emap_sgRNA, singlesTable):
    emap_gene = pd.DataFrame(np.zeros((len(set(singlesTable['gene'])),len(set(singlesTable['gene'])))), index = sorted(set(singlesTable['gene'])), columns = sorted(set(singlesTable['gene'])))
    for gene_a, rowgroup in emap_sgRNA.groupby(singlesTable['gene']):
        for gene_b, colgroup in rowgroup.groupby(singlesTable['gene'], axis=1):
            emap_gene.loc[gene_a, gene_b] = colgroup.sum().sum() / (colgroup.shape[0] * colgroup.shape[1])
            
    return emap_gene

### fit functions for calculating interactions and plotting
def linearFitForceIntercept(xdata, ydata, bdata):
    m1 = optimize.fmin(lambda m, x, y: ((m*x + bdata - y)**2).sum(), x0=0.1, args=(xdata, ydata), disp=0)[0]
    
    return lambda x1: m1*np.array(x1) + bdata

def quadFitForceIntercept(xdata, ydata, bdata):
    m1 = optimize.fmin(lambda m, x, y: ((m[0]*(x**2) + m[1]*x + bdata - y)**2).sum(), x0=[0.1,0.1], args=(xdata, ydata), disp=0)
    
    return lambda x1: m1[0]*(np.array(x1)**2) + m1[1]*np.array(x1) + bdata


In [12]:
## Import data as DataFrames
## relevant files can be found here: https://ucsf.box.com/s/nuwov4kgb55mqfrr7j5a216ot78f4uxn

## "countsFile" is to be defined using the file titled "GIscreen_countsMasterTable.txt" 
## "doublesLibrary" is to be defined using the file titled "GIscreen_dualsgrna_aligned_library.txt"

countsFile = pd.read_csv('',sep='\t',index_col=0)
doublesLibrary = pd.read_csv('',sep='\t',index_col=0)



In [15]:
## Define a dictionary "growthScores" that has as keys the different replicates and conditions of the experiment,
## where each key's value corresponds to the number of population doublings that occurred in the experiment, also
## include the difference between replicate matched experiments

growthScores = {
    'rep1_growth' : 9.632926829,
    'rep1_diff' : 4.87195122,
    'rep1_treated' : 4.76097561,
    'rep2_growth' : 9.559756098,
    'rep2_diff' : 4.752439024,
    'rep2_treated' : 4.807317073}

In [16]:
##Calculate normalized log fold enrichment scores.

#For Gamma, first arg is "RepX_T0", second arg is "RepX_UT, doublingDifferences is "repX_growth"
#For Tau, first arg is "RepX_T0", second arg is "RepX_DRUG, doublingDifferences is "repX_treated"
#For Rho, first arg is "RepX_UT", second arg is "RepX_DRUG, doublingDifferences is "repX_diff"


## Calculate sgRNA level tau scores

taulog2es_rep1 = calcLog2e_cycledonly(countsFile['Rep1_T0'],
                                   countsFile['Rep1_DRUG'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['rep1_treated'])

taulog2es_rep2 = calcLog2e_cycledonly(countsFile['Rep2_T0'],
                                   countsFile['Rep2_DRUG'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['rep2_treated'])



370881 pairs of 407044 passing filter
357604 pairs of 407044 passing filter


In [17]:
## Calculate sgRNA level gamma scores

gammalog2es_rep1 = calcLog2e_cycledonly(countsFile['Rep1_T0'],
                                   countsFile['Rep1_UT'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['rep1_growth'])

gammalog2es_rep2 = calcLog2e_cycledonly(countsFile['Rep2_T0'],
                                   countsFile['Rep2_UT'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['rep2_growth'])

400689 pairs of 407044 passing filter
398161 pairs of 407044 passing filter


In [18]:
## Repave taus replicates to have the same list of threshold passed sgRNAs, then average sgRNA phenotype across reps

sgIntersect =set(taulog2es_rep1.index).intersection(taulog2es_rep2.index)
tauLog2es_repave = ((taulog2es_rep1 + taulog2es_rep2) / 2).loc[sgIntersect]
print(len(tauLog2es_repave))

357604


In [19]:
## Repave gammas replicates to have the same list of threshold passed sgRNAs, then average sgRNA phenotype across reps

sgIntersect =set(gammalog2es_rep1.index).intersection(gammalog2es_rep2.index)
gammaLog2es_repave = ((gammalog2es_rep1 + gammalog2es_rep2) / 2).loc[sgIntersect]
print(len(gammaLog2es_repave))

398161


In [20]:
## Repave both the tau and gamma replicate averaged as well as individual replicates to all have the same indices

bothIntersect = set(gammaLog2es_repave.index).intersection(tauLog2es_repave.index)
gammaLog2es_repave = gammaLog2es_repave.loc[bothIntersect]
tauLog2es_repave = tauLog2es_repave.loc[bothIntersect]
gammalog2es_rep1 = gammalog2es_rep1.loc[bothIntersect]
gammalog2es_rep2 = gammalog2es_rep2.loc[bothIntersect]
taulog2es_rep1 = taulog2es_rep1.loc[bothIntersect]
taulog2es_rep2 = taulog2es_rep2.loc[bothIntersect]

print(len(tauLog2es_repave))

354025


In [21]:
## Create replicate averaged eGI map from the Tau data 

## First put dual sgRNA taus into matrix format, make DataFrames of single sgRNA taus
tauPhenotypeMatrix, tauSinglesTable, tauSinglePhenotypes = generatePhenotypeMatrix(tauLog2es_repave)

## ABBA normalize (both possible orientations of a paired sgRNA)
tau_phenotypeMatrix_abba, tau_singlePhenotypes_abba = abbaAveragePhenotypes(tauPhenotypeMatrix, tauSinglesTable)

## Calculate sgRNA level genetic interactions
tau_emap1, tau_emap2, tau_emap_quad_std = calculateInteractions(tau_phenotypeMatrix_abba, tau_singlePhenotypes_abba, tauSinglesTable, quadFitForceIntercept, zstandardize=True)

## Calculate gene level genetic interactions, this is used to make Table S4
tau_emap_quad_std_gene = generateGeneMap(tau_emap_quad_std, tauSinglesTable)


In [ ]:
## Create replicate specific GI maps from the Tau data

## First put dual sgRNA tau scores into matrix format
tau_phenotypeMatrix_rep1, tau_singlesTable_rep1, tau_singlePhenotypes_rep1 = generatePhenotypeMatrix(taulog2es_rep1)
tau_phenotypeMatrix_rep2, tau_singlesTable_rep2, tau_singlePhenotypes_rep2 = generatePhenotypeMatrix(taulog2es_rep2)

## ABBA normalize
tau_phenotypeMatrix_rep1_abba, tau_singlePhenotypes_rep1_abba = abbaAveragePhenotypes(tau_phenotypeMatrix_rep1, tau_singlesTable_rep1)
tau_phenotypeMatrix_rep2_abba, tau_singlePhenotypes_rep2_abba = abbaAveragePhenotypes(tau_phenotypeMatrix_rep2, tau_singlesTable_rep1)

## Calculate sgRNA level genetic interactions 
emap1, emap2, tau_emap_quad_std_rep1 = calculateInteractions(tau_phenotypeMatrix_rep1_abba, tau_singlePhenotypes_rep1_abba, tau_singlesTable_rep1, quadFitForceIntercept, zstandardize=True)
emap1, emap2, tau_emap_quad_std_rep2 = calculateInteractions(tau_phenotypeMatrix_rep2_abba, tau_singlePhenotypes_rep2_abba, tau_singlesTable_rep2, quadFitForceIntercept, zstandardize=True)

## Calculate gene level genetic interactions
tau_emap_quad_std_rep1_gene = generateGeneMap(tau_emap_quad_std_rep1, tau_singlesTable_rep1)
tau_emap_quad_std_rep2_gene = generateGeneMap(tau_emap_quad_std_rep2, tau_singlesTable_rep2)

In [ ]:
## Create replicate averaged GI map from the Gamma data 

## First put dual sgRNA gammas into matrix format, make DataFrames of single sgRNA gammas
gammaPhenotypeMatrix, gammaSinglesTable, gammaSinglePhenotypes = generatePhenotypeMatrix(gammaLog2es_repave)

## ABBA normalize
gamma_phenotypeMatrix_abba, gamma_singlePhenotypes_abba = abbaAveragePhenotypes(gammaPhenotypeMatrix, gammaSinglesTable)

## Calculate sgRNA level genetic interactions
emap1, emap2, gamma_emap_quad_std = calculateInteractions(gamma_phenotypeMatrix_abba, gamma_singlePhenotypes_abba, gammaSinglesTable, quadFitForceIntercept, zstandardize=True)

## Calculate gene level genetic interactions, this is used to make Table S3
gamma_emap_quad_std_gene = generateGeneMap(gamma_emap_quad_std, gammaSinglesTable)


In [ ]:
## Create replicate specific GI maps from the Gamma data

## First put paired normalized log enrichment scores into a matrix, make DataFrames of singles scores and sgRNAs
gamma_phenotypeMatrix_rep1, gamma_singlesTable_rep1, gamma_singlePhenotypes_rep1 = generatePhenotypeMatrix(gammalog2es_rep1)
gamma_phenotypeMatrix_rep2, gamma_singlesTable_rep2, gamma_singlePhenotypes_rep2 = generatePhenotypeMatrix(gammalog2es_rep2)

## ABBA normalize this data
gamma_phenotypeMatrix_rep1_abba, gamma_singlePhenotypes_rep1_abba = abbaAveragePhenotypes(gamma_phenotypeMatrix_rep1, gamma_singlesTable_rep1)
gamma_phenotypeMatrix_rep2_abba, gamma_singlePhenotypes_rep2_abba = abbaAveragePhenotypes(gamma_phenotypeMatrix_rep2, gamma_singlesTable_rep1)

## Calculate sgRNA level genetic interactions 
emap1, emap2, gamma_emap_quad_std_rep1 = calculateInteractions(gamma_phenotypeMatrix_rep1_abba, gamma_singlePhenotypes_rep1_abba, gamma_singlesTable_rep1, quadFitForceIntercept, zstandardize=True)
emap1, emap2, gamma_emap_quad_std_rep2 = calculateInteractions(gamma_phenotypeMatrix_rep2_abba, gamma_singlePhenotypes_rep2_abba, gamma_singlesTable_rep2, quadFitForceIntercept, zstandardize=True)

## Calculate gene level genetic interactions
gamma_emap_quad_std_rep1_gene = generateGeneMap(gamma_emap_quad_std_rep1, gamma_singlesTable_rep1)
gamma_emap_quad_std_rep2_gene = generateGeneMap(gamma_emap_quad_std_rep2, gamma_singlesTable_rep2)